# Movielens 영화 추천

# Step1. Import Library

In [47]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import tensorflow as tf
import scipy
import implicit

# Step2. Import Data
이번에 활용할 데이터셋은 추천시스템의 MNIST라고 부를만한 Movielens입니다. 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기별로 있습니다.\
별점 데이터는 대표적인 explicit데이터이지만 implicit데이터로 간주하고 테스트해 볼 수 있습니다. 여기서는 별점을 시청횟수로 해석해서 생각하겠습니다.

In [48]:
# 별점 데이터를 불러옵니다.
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [49]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


# Step3. Data Preprocessing
유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

In [50]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [51]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

# timestamp도 필요없으니 제거하겠습니다.
ratings = ratings[['user_id','movie_id','counts']]

ratings.head()

,user_id,movie_id,counts
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


검색을 쉽게하기위해 영화제목과 장르를 전부 소문자로 바꿔줍니다.

In [52]:
movies['title'] = movies['title'].str.lower()
movies['genre'] = movies['genre'].str.lower()
movies.head()

,movie_id,title,genre
0,1,toy story (1995),animation|children's|comedy
1,2,jumanji (1995),adventure|children's|fantasy
2,3,grumpier old men (1995),comedy|romance
3,4,waiting to exhale (1995),comedy|drama
4,5,father of the bride part ii (1995),comedy


결측치나 중복치가 있는지 확인해줍니다.

In [53]:
print('ratings 결측치 수:',ratings.isnull().sum())
print('movies 결측치 수:', movies.isnull().sum())

ratings 결측치 수: user_id     0
movie_id    0
counts      0
dtype: int64
movies 결측치 수: movie_id    0
title       0
genre       0
dtype: int64


In [54]:
# 장르는 중복일 수 있어도 영화 이름이 중복이면 안됩니다. 
print('movies의 중복치 수 :',movies['title'].duplicated().sum())

movies의 중복치 수 : 0


# Step4. Exploratory Data Analysis (EDA)

In [55]:
print('유저수 :', ratings['user_id'].nunique())
print('영화수 :', movies['title'].nunique())

유저수 : 6039
영화수 : 3883


인기순으로 영화목록 30개를 출력해보겠습니다.

In [56]:
# 영화id별 유저id수를 movie_count변수에 넣습니다.
movie_count = ratings.groupby('movie_id')['user_id'].count()

# 내림차순으로 30개를 나열하고 해당 영와id를 넘파이 배열형태로 mv_id_30변수에 저장합니다.
mv_id_30 = movie_count.sort_values(ascending=False)[:30].index.values

# loc를 이용해서 id에 해당하는 영화 이름을 출력합니다.
movies.loc[mv_id_30,'title']

2858                               brief encounter (1946)
260                              ladybird ladybird (1994)
1196                                         alien (1979)
1210                                   raging bull (1980)
2028               something wicked this way comes (1983)
589                      silence of the lambs, the (1991)
593                                   pretty woman (1990)
1198                 big blue, the (le grand bleu) (1988)
1270                        some kind of wonderful (1987)
2571                                      superman (1978)
480                                         lassie (1994)
2762                            dog of flanders, a (1999)
608                                pallbearer, the (1996)
110                            rumble in the bronx (1995)
1580                                    wishmaster (1997)
527                             secret garden, the (1993)
1197                              army of darkness (1993)
2396          

우리는 본인의 영화취향과 가장 유사한 또다른 영화를 추천받고 싶습니다. 넷플릭스 등 추천 시스템들은 이를 위해서 처음 가입하는 사용자의 취향인 영화정보를 5개 이상 입력받는 과정을 거치게 하는 경우가 많습니다.\
이와 동일한 과정을 위해 위 데이터셋에 제가 좋아하는 영화5개를 추가하겠습니다. 

In [57]:
# alien, toy story, superman, heathers, priest
my_favorite = [1196, 1, 2571, 1265, 296]

# 제 자신의 id를 9999로 설정하고, 각 영화에 대한 별점은 5점을 주겠습니다.
my_list = pd.DataFrame({'user_id': [9999]*5, 'movie_id': my_favorite, 'counts': [5]*5})
ratings = ratings.append(my_list)
ratings.tail(10)

,user_id,movie_id,counts
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4
1000208,6040,1097,4
0,9999,1196,5
1,9999,1,5
2,9999,2571,5
3,9999,1265,5
4,9999,296,5


## 모델에 활용하기 위한 전처리
사람이 태어나면 주민등록번호가, 학교에 가면 출석번호가 있듯이 데이터의 관리를 쉽게 하기 위해 번호를 붙여주고 싶습니다. 우리가 다루는 데이터에서는 user와 movie 각각에 번호를 붙이고 싶습니다. 보통 이런 작업을 indexing이라고 합니다.

In [58]:
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [59]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 movie_id 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie_id column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('movid_id column indexing Fail!!')

ratings

user_id column indexing OK!!
movie_id column indexing OK!!


,user_id,movie_id,counts
0,0,0,5
1,0,1,3
2,0,2,3
3,0,3,4
4,0,4,5
...,...,...,...
0,6039,117,5
1,6039,40,5
2,6039,124,5
3,6039,110,5


# Step5. CSR matrix
CSR Matrix는 Sparse한 matrix에서 0이 아닌 유효한 데이터로 채워지는 데이터의 값과 좌표 정보만으로 구성하여 메모리 사용량을 최소화하면서도 Sparse한 matrix와 동일한 행렬을 표현할 수 있도록 하는 데이터 구조입니다.

CSR matrix를 직접 만들어 봅시다.

In [60]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_ratings = csr_matrix((ratings.counts, (ratings.user_id, ratings.movie_id)), shape=(num_user, num_movie))
csr_ratings

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

# Step6. Model training 
MF모델을 implicit패키지를 사용하여 학습해봅시다.\
implicit패키지는 암묵적 데이터셋을 사용하는 다양한 모델을 굉장히 빠르게 학습할 수 있는 패키지입니다.\
이 패키지에 구현된 als(AlternatingLeastSquares)모델을 사용하겠습니다. MF에서 쪼개진 두 feature matrix를 한꺼번에 훈련하는 것은 잘 수렴하지 않기 때문에, 한쪽을 고정시키고 다른쪽을 학습하는 방식을 번갈아 수행하는 ALS방식이 효과적인 것으로 알려져있습니다.

In [61]:
from implicit.als import AlternatingLeastSquares

# implicit 라이브러리에서 권장하고 있는 부분입니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

ALS클래스의 \__init\__파라미터를 살펴보겠습니다.
- factors : 유저와 아이템의 벡터를 몇차원으로 할 것인지
- regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
- use_gpu : GPU를 사용할 것인지
- iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지

In [62]:
als_model = AlternatingLeastSquares(factors=300, regularization=0.01, use_gpu=False, iterations=40, dtype=np.float32)

In [63]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_ratings_transpose = csr_ratings.T
csr_ratings_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [64]:
als_model.fit(csr_ratings_transpose)

  0%|          | 0/40 [00:00<?, ?it/s]

학습이 끝났습니다. 이제 2가지를 살펴보도록하겠습니다.
- Zimin벡터와 black eyed peas의 벡터를 어떻게 만들고 있는지
- 두 벡터를 곱하면 어떤 값이 나오는지

In [65]:
me, alien = user_to_idx[9999], movie_to_idx[1196]
me_vector, alien_vector = als_model.user_factors[me], als_model.item_factors[alien]

In [66]:
me_vector

array([ 0.30016473,  0.86023206,  0.28243   , -0.13589962, -0.5916186 ,
        0.103519  , -0.252378  ,  0.1679542 , -0.06745457, -0.53675854,
       -0.08422491, -0.26791048,  0.11594322, -0.4080301 ,  0.06285167,
        0.24692184,  0.14361449,  0.13082552,  0.10392639, -0.3475649 ,
        0.1496592 ,  0.4874466 , -0.21825401, -0.14976439, -0.17888671,
        0.22411948, -0.2630345 , -0.04123054,  0.00807257, -0.84748596,
       -0.3389729 , -0.19131483, -0.42047983,  0.3207705 ,  0.6878367 ,
        0.0092933 ,  0.6273585 , -0.16142893, -0.06481932,  0.12175745,
       -0.10274427, -0.12433131,  0.0264738 , -0.15363047,  0.18653437,
        0.22367871,  0.12474412, -0.29000443, -0.1400519 ,  0.05307795,
       -0.537913  ,  0.23570421, -0.05009994, -0.09181053, -0.06583971,
        0.20212875,  0.42560038,  0.26548463, -0.09983342, -0.25656366,
       -0.10327875, -0.4320942 , -0.67171013, -0.0099426 , -0.5236291 ,
        0.2701663 ,  0.40647787, -0.2793835 , -0.23221052,  0.29

In [67]:
alien_vector

array([-0.00758583,  0.05695585,  0.0063816 , -0.03024415,  0.00701888,
        0.0225992 , -0.01648337,  0.03518625,  0.00804768, -0.0329825 ,
        0.00858269, -0.01891775, -0.00674972, -0.0064504 ,  0.0343454 ,
        0.04292023,  0.02080316,  0.01032652,  0.01341076,  0.02389689,
        0.01652422,  0.01854838,  0.02227536,  0.02455162,  0.00255307,
       -0.00799834,  0.00679038, -0.0106111 ,  0.0226893 , -0.03542295,
       -0.03083599, -0.02127305, -0.01611517,  0.02279817,  0.00267603,
        0.04125925,  0.02670424,  0.0225335 , -0.01867661,  0.01466633,
        0.02608223, -0.01443455, -0.00643171, -0.01170246,  0.00624209,
        0.01591256, -0.00150732,  0.02217448,  0.01416037,  0.01318167,
       -0.02777864,  0.03743947, -0.0085997 ,  0.02588191, -0.00610117,
        0.00313236,  0.02804427,  0.02505186,  0.02460274,  0.02151442,
       -0.00131431, -0.00947249, -0.01305727,  0.00916569, -0.03280994,
        0.02386141,  0.02039376, -0.00231506, -0.00856794,  0.01

In [68]:
# me와 alien을 내적하는 코드
np.dot(me_vector, alien_vector)

0.8131687

선호도가 꽤 높게 나왔습니다. 잘 훈련된 것 같습니다.\
이렇게 학습된 모델을가지고 toy story에 대한 선호도를 어떻게 예측할지 한 번 보겠습니다.\
참고로 toy story는 이전에 제가 직접 별점5개를 준 영화입니다.

In [69]:
# movie_to_idx[1]은 toy story를 뜻합니다.
toy_story = movie_to_idx[1]
toy_vector = als_model.item_factors[toy_story]
np.dot(me_vector, toy_vector)

0.8558811

역시나 선호도가 높게나왔네요

# Step7. 내가 좋아하는 영화와 비슷한 영화를 추천받기
ALS클래스에 구현되어 있는 similar_items메서드를 통하여 비슷한 영화를 찾습니다. 처음으로는 제가 좋아하는 alien으로 찾아보겠습니다.

In [70]:
favorite_movie_idx = 1196 # alien 인덱스
movie_id = movie_to_idx[favorite_movie_idx]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(117, 1.0000001),
 (44, 0.4457005),
 (64, 0.37535822),
 (651, 0.2669032),
 (3487, 0.26422304),
 (2763, 0.26382694),
 (2030, 0.26351228),
 (1993, 0.26215544),
 (1056, 0.25919038),
 (3424, 0.25880596),
 (2038, 0.25811845),
 (3268, 0.25506988),
 (3011, 0.25441882),
 (3493, 0.25355297),
 (120, 0.25167042)]

(movie의 id, 유사도)Tuple을 반환하고있습니다. 하지만 가장 유사한 영화의 유사도가 0.4정도밖에 안되네요. 그래도 일단 movie의 id를 다시 movie의 title로 매핑 시켜주겠습니다.

In [71]:
# movie_to_idx를 뒤집어, index로부터 movie이름을 얻는 dict를 생성합니다.
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
mv_id = [idx_to_movie[i[0]] for i in similar_movie]
mv_id

[1196,
 260,
 1210,
 1200,
 3374,
 1471,
 3282,
 885,
 771,
 212,
 84,
 1174,
 56,
 3595,
 1198]

In [72]:
movies['title'][mv_id]

1196                                 alien (1979)
260                      ladybird ladybird (1994)
1210                           raging bull (1980)
1200    killer, the (die xue shuang xiong) (1989)
3374                         born american (1986)
1471               8 heads in a duffel bag (1997)
3282                 two thousand maniacs! (1964)
885                for whom the bell tolls (1943)
771                        stealing beauty (1996)
212         before the rain (pred dozhdot) (1994)
84                      angels and insects (1995)
1174                madonna: truth or dare (1991)
56                   home for the holidays (1995)
3595    puppet master 5: the final chapter (1994)
1198         big blue, the (le grand bleu) (1988)
Name: title, dtype: object

alien과 유사도가 0.4인 영화가 ladybird ladybird...

# Step8.  내가 가장 좋아할 만한 영화들을 추천받기
ALS클래스에 구현되어 있는 recommend메서드를 통하여 제가 좋아할 만한 영화를 추천받습니다.\
filter_already_liked_items는 유저가 이미 평가한 아이템은 제외하는 Argument입니다.

In [73]:
user = user_to_idx[9999]

#recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_ratings, N=20, filter_already_liked_items=True)
movie_recommended

[(44, 0.35678098),
 (64, 0.3207865),
 (269, 0.21307306),
 (157, 0.2111229),
 (92, 0.20462486),
 (50, 0.19790176),
 (48, 0.16592391),
 (289, 0.15750214),
 (120, 0.15068266),
 (613, 0.14807896),
 (431, 0.14635257),
 (369, 0.14463088),
 (5, 0.13911499),
 (22, 0.1366055),
 (62, 0.13071413),
 (288, 0.13046026),
 (233, 0.12835252),
 (651, 0.12463674),
 (38, 0.122327514),
 (476, 0.12049795)]

해당 인덱스들을 영화id로 바꿔봅시다.

In [74]:
[idx_to_movie[i[0]] for i in movie_recommended]

[260,
 1210,
 1213,
 318,
 589,
 3114,
 2028,
 1089,
 1198,
 357,
 1653,
 1674,
 1197,
 1270,
 2916,
 1517,
 50,
 1200,
 2762,
 1148]

ladybird를 추천해주고있습니다. 모델은 왜 ladybird를 추천해줬을까요? als클래스에 구현된 explain메서드를 사용해 제가 기록을 남긴 데이터중 이 추천에 기여한 정도를 확인할 수 있습니다.

In [75]:
ladybird = movie_to_idx[1]
explain = als_model.explain(user, csr_ratings, itemid=ladybird)

이 method는 추천한 콘텐츠의 점수에 기여한 다른 콘텐츠와 기여도(합이 콘텐츠의 점수가 됩니다.)를 반환합니다. 어떤 영화들이 이 추천에 얼마나 기여하고 있는걸까요?

In [76]:
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[(1, 0.7928939808026867),
 (2571, 0.03235169660420819),
 (1265, 0.02484789814219636),
 (296, 0.0066136355758636065),
 (1196, -0.008743442240435394)]

토이스토리의 기여가 0.79로 대부분입니다.(index:1)
그런데 결과가 조금 이상합니다. filter_already_liked_items=True로 했는데 이미 평가한 아이템이 나왔습니다. 이것은 implicit 버전 0.4.2에서 생긴 버그라고합니다.

# 회고
- 이번 실습에서 추천시스템을 구현했는데 implicit라이브러리의 코랩과 노드의 버전차이로인한 오류가 많이 생겼었다. 그래서 als모델에 csr_ratings를 넣어줄때 트랜스포즈를 취해줘야하는데, 오히려 트랜스포즈를 취해주면 모델의 user_factors와 item_factors이 반대로 값이 나와버려서 트랜스포즈를 지워줬어야했다.
- 버전차이때문에 모델의 similar_items도 이상하게나왔었다. 영화의id와 유사도가 한쌍으로 출력이되어야 정상인데, 코랩에서는 id로된 배열과 유사도로이루어진 배열이 따로따로 나왔었다. 그러니 당연히 기존코드도 변경된 배열에따른 수정을 해줘야했다.
- 그리고 als의 recommend를 실행해보니 오류가났었다. 여기서 시간을 제일많이 끌었다.. 결국 aiffel주피터노트북으로 해보니 이전에 있던 오류들까지 전부 정상으로 실행되었고 ,다른이 하는거 봤는데 코랩에서 노드와 똑같이 버전을 맞추니 정상적으로 실행되는걸보니 결국 버전땜에 발생한 오류인걸 알아냈다.